In [12]:
import pandas as pd
import plotly.express as px
import numpy as np
import re
import math as mth
from .. import cleaning_text

ImportError: attempted relative import with no known parent package

In [ ]:
# Read and save dataset into 
cupid_df = pd.read_csv("../data/okcupid_profiles.csv")

In [ ]:
# Sampling the rows
cupid_df.sample(5)

,age,status,sex,orientation,body_type,diet,drinks,drugs,education,ethnicity,...,essay0,essay1,essay2,essay3,essay4,essay5,essay6,essay7,essay8,essay9
11886,35,available,f,bisexual,NaN,strictly vegetarian,rarely,NaN,graduated from college/university,NaN,...,i'm opinionated and enthusiastic about many th...,i'm a professional costumer and corsetier.,foxtrot and viennese waltz. i'm a walking the...,my hair is more than half my height.,books: last chance to see (his underrated mast...,feminism.,politics. it drives me nuts how conservative t...,playing rock band. i hit things with sticks.,"i had my tubes tied (burned shut with a laser,...",you want to start a conversation. i don't resp...
17669,24,single,m,gay,thin,mostly vegan,socially,never,graduated from masters program,"hispanic / latin, other",...,"i'm lactose intolerant.. sooo, that's pretty s...","before san francisco, i was living in panama d...",...eating my body weight in hummus i make sur...,i'm kind of a ginger...ish.. even more rare th...,most of my favorite books are the books i hate...,NaN,NaN,dancing like i know what i am doing.,NaN,...you have a sense of humor about yourself.
33676,58,single,m,straight,thin,mostly vegetarian,not at all,never,NaN,white,...,"i'm slim, have blue eyes and glasses, am quirk...",i'm director of a independent company and work...,being intuitive. i read people well. i'm also ...,that i'm outgoing and vivacious (though i'm mu...,i've decided not to try and match myself with ...,my spiritual practice my sexuality cinema indi...,"new creative projects, why i'm alive, why peop...",attending eclectic art and media events in sf....,i'm emotionally available.,you're interested in talking and meeting someo...
4286,59,single,f,straight,thin,mostly anything,socially,never,graduated from college/university,white,...,in her own words dichotomy - that's me. i am ...,NaN,NaN,"my smile, my warmth, my personal style, my hair",all over the place: food: wild kitchen dinners...,"friends, great conversation, zinfandel, chocol...",being present,"there is never a typicalfriday night for me, a...",will be revealed to you fairly quickly if i kn...,my profile tickles your fancy you are emotiona...
48204,31,single,m,straight,athletic,NaN,socially,never,graduated from ph.d program,white,...,i seem to be attracting the wrong sort of prop...,making and keeping friends from varied walks o...,anything mathematical and spatial. out-thinkin...,NaN,NaN,NaN,"human social dynamics, the reasons we behave t...",busy.,NaN,NaN


In [ ]:
# Check dataset data types
cupid_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59946 entries, 0 to 59945
Data columns (total 31 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   age          59946 non-null  int64  
 1   status       59946 non-null  object 
 2   sex          59946 non-null  object 
 3   orientation  59946 non-null  object 
 4   body_type    54650 non-null  object 
 5   diet         35551 non-null  object 
 6   drinks       56961 non-null  object 
 7   drugs        45866 non-null  object 
 8   education    53318 non-null  object 
 9   ethnicity    54266 non-null  object 
 10  height       59943 non-null  float64
 11  income       59946 non-null  int64  
 12  job          51748 non-null  object 
 13  last_online  59946 non-null  object 
 14  location     59946 non-null  object 
 15  offspring    24385 non-null  object 
 16  pets         40025 non-null  object 
 17  religion     39720 non-null  object 
 18  sign         48890 non-null  object 
 19  smok

In [ ]:
# Duplicated rows
cupid_df.duplicated().sum()

0

In [ ]:
# Null cells
cupid_df.isnull().sum()

age                0
status             0
sex                0
orientation        0
body_type       5296
diet           24395
drinks          2985
drugs          14080
education       6628
ethnicity       5680
height             3
income             0
job             8198
last_online        0
location           0
offspring      35561
pets           19921
religion       20226
sign           11056
smokes          5512
speaks            50
essay0          5488
essay1          7572
essay2          9638
essay3         11476
essay4         10537
essay5         10850
essay6         13771
essay7         12451
essay8         19225
essay9         12603
dtype: int64

In [ ]:
cupid_df = cupid_df.fillna('')
cupid_df.isnull().sum()

age            0
status         0
sex            0
orientation    0
body_type      0
diet           0
drinks         0
drugs          0
education      0
ethnicity      0
height         0
income         0
job            0
last_online    0
location       0
offspring      0
pets           0
religion       0
sign           0
smokes         0
speaks         0
essay0         0
essay1         0
essay2         0
essay3         0
essay4         0
essay5         0
essay6         0
essay7         0
essay8         0
essay9         0
dtype: int64

In [ ]:
# 
cupid_df.describe()

,age,income
count,59946.000000,59946.000000
mean,32.340290,20033.222534
std,9.452779,97346.192104
min,18.000000,-1.000000
25%,26.000000,-1.000000
50%,30.000000,-1.000000
75%,37.000000,-1.000000
max,110.000000,1000000.000000


In [ ]:
# Count the number of  unique values
cupid_df.nunique()

age               54
status             5
sex                2
orientation        3
body_type         13
diet              19
drinks             7
drugs              4
education         33
ethnicity        218
height            61
income            13
job               22
last_online    30123
location         199
offspring         16
pets              16
religion          46
sign              49
smokes             6
speaks          7648
essay0         54348
essay1         51517
essay2         48626
essay3         43521
essay4         49258
essay5         48962
essay6         43584
essay7         45549
essay8         39324
essay9         45441
dtype: int64

In [ ]:
# Seeing/counting all the labels
for c in cupid_df.columns:    
    print (f"----< {c} >----")
    print(cupid_df[c].value_counts())

----< age >----
26     3724
27     3685
28     3583
25     3531
29     3295
24     3242
30     3149
31     2735
23     2592
32     2587
33     2206
22     1934
34     1902
35     1755
36     1583
37     1427
38     1330
21     1282
39     1172
42     1072
40     1030
41      980
20      953
43      858
44      708
45      643
19      611
46      578
47      529
48      481
49      459
50      437
51      350
52      344
18      309
56      271
54      267
55      265
57      256
53      252
59      221
58      197
60      195
61      176
62      167
63      138
64      113
65      109
66      105
67       66
68       59
69       31
110       1
109       1
Name: age, dtype: int64
----< status >----
single            55697
seeing someone     2064
available          1865
married             310
unknown              10
Name: status, dtype: int64
----< sex >----
m    35829
f    24117
Name: sex, dtype: int64
----< orientation >----
straight    51606
gay          5573
bisexual     2767
Name: 

In [ ]:
# Accounting for number of unique types of each categorical columns
# This unq_val_count Series will be used for String look up through <essayX> columns
unq_val_count={c:pd.DataFrame(cupid_df[c].value_counts()) for c in cupid_df.columns}
unq_val_count = pd.Series(unq_val_count)
unq_val_count

age                  age
26   3724
27   3685
28   3583
25   3...
status                         status
single           55697
...
sex                                        sex
m  35829
f  24117
orientation              orientation
straight        51606
ga...
body_type                      body_type
average             ...
diet                                 diet
                   ...
drinks                      drinks
socially      41780
rarely...
drugs                     drugs
never      37724
           1...
education                                         education
g...
ethnicity                                                    ...
height               height
70.0    6074
68.0    5449
67.0   ...
income                   income
-1         48442
 20000      ...
job                                                job
      ...
last_online                      last_online
2012-06-29-22-56...
location                                       location
san f...
offspring                

In [ ]:
# age               54, sex                2, drinks             6, drugs              3, education         32, ethnicity        217

orientation_df = cupid_df['orientation'].value_counts()
fig=px.treemap(orientation_df,
               path=[orientation_df.index],
               values=orientation_df.values,
               title="Orientation distribution",
               color=orientation_df.index,
               color_discrete_sequence=px.colors.sequential.PuBuGn,
               template='plotly_dark',
               width=1000, height=500
              )
percents = np.round((100*orientation_df.values / sum(orientation_df.values)).tolist(),2)
fig.data[0].customdata = [percents[0],percents[1],percents[2]]
fig.data[0].texttemplate = '%{label}<br>%{value}<br>%{customdata}%'
fig.update_layout(font=dict(size=19,family="Franklin Gothic"))

fig.show()

/opt/anaconda3/lib/python3.9/site-packages/plotly/express/_core.py:1637: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_all_trees = df_all_trees.append(df_tree, ignore_index=True)


In [ ]:
# age               54, sex                2, drinks             6, drugs              3, education         32, ethnicity        217

gender_df = cupid_df['sex'].value_counts()
fig=px.treemap(gender_df,
               path=[gender_df.index],
               values=gender_df.values,
               title="Gender distribution",
               color=gender_df.index,
               color_discrete_sequence=px.colors.sequential.PuBuGn,
               template='plotly_dark',
               width=1000, height=500
              )

percents = np.round((100*gender_df.values / sum(gender_df.values)).tolist(),2)
fig.data[0].customdata = percents
fig.data[0].texttemplate = '%{label}<br>%{value}<br>%{customdata}%'
fig.update_layout(font=dict(size=19,family="Franklin Gothic"))
fig.show()

/opt/anaconda3/lib/python3.9/site-packages/plotly/express/_core.py:1637: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.



In [ ]:
age_df=cupid_df['age'].value_counts()
fig=px.treemap(age_df,
               path=[age_df.index],
               values=age_df.values,
               title="Age distribution",
               color=age_df.index,
               color_discrete_sequence=px.colors.sequential.PuBuGn,
               template='plotly_dark',
               width=1000, height=500
              )

percents = np.round((100*age_df.values / sum(age_df.values)).tolist(),2)
fig.data[0].customdata = percents
fig.data[0].texttemplate = '%{label}<br>%{value}<br>%{customdata}%'
fig.update_layout(font=dict(size=19,family="Franklin Gothic"))
fig.show()

/opt/anaconda3/lib/python3.9/site-packages/plotly/express/_core.py:1637: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.



In [ ]:
# Revised education column, using custom created csv 
# through self labeling using the most conservative judgement
edu_cat_df = pd.read_csv("../data/education_category_revised.csv")
edu_cat_df

,original,revised
0,graduated from college/university,Associate's Degree
1,graduated from masters program,Master's Degree
2,working on college/university,Some College
3,working on masters program,Bachelor's Degree
4,graduated from two-year college,Associate's Degree
5,graduated from high school,High school
6,graduated from ph.d program,Doctorate Degree
7,graduated from law school,Law degree
8,working on two-year college,Some College
9,dropped out of college/university,Some College


In [ ]:
# Prepping data to use with .replace()
edu_cat_df = edu_cat_df.set_index('original')
edu_cat_df.to_dict()

{'revised': {'graduated from college/university': "Associate's Degree",
  'graduated from masters program': "Master's Degree",
  'working on college/university': 'Some College',
  'working on masters program': "Bachelor's Degree",
  'graduated from two-year college': "Associate's Degree",
  'graduated from high school': 'High school',
  'graduated from ph.d program': 'Doctorate Degree',
  'graduated from law school': 'Law degree',
  'working on two-year college': 'Some College',
  'dropped out of college/university': 'Some College',
  'working on ph.d program': "Master's Degree",
  'college/university': "Associate's Degree",
  'graduated from space camp': 'Space Camp',
  'dropped out of space camp': 'Some Education',
  'graduated from med school': 'Medical Degree',
  'working on space camp': 'Some Education',
  'working on law school': "Bachelor's Degree",
  'two-year college': "Associate's Degree",
  'working on med school': "Bachelor's Degree",
  'dropped out of two-year college': 'S

In [ ]:
speak_df = cupid_df['speaks']
for i in speak_df:
    print(i)

english
english (fluently), spanish (poorly), french (poorly)
english, french, c++
english, german (poorly)
english
english (fluently), chinese (okay)
english
english, spanish (okay)
english
english (fluently)
english
english (fluently), sign language (poorly)
english
english
english
english (fluently), spanish (okay)
english (fluently), c++ (fluently), german (poorly), french (poorly)
english
english (fluently)
english (fluently)
english (fluently)
english (fluently), spanish (fluently), italian (okay)
english, spanish (poorly)
english
english
english
english (fluently), tagalog (fluently), japanese (okay)
english (fluently), other (fluently), thai (poorly)
english, spanish (poorly)
english
english, spanish (poorly)
english
english
english (fluently), spanish (fluently)
english
english, russian
english (fluently), spanish (fluently)
english (fluently), spanish (poorly)
english
english
english (fluently), dutch (fluently), lisp (fluently), spanish (okay), german (okay)
english
english,

In [ ]:
# Picking just columns speaks
speak_df = cupid_df['speaks']
# Creating empty list to store ID of someone who speak Vietnamese
vietnamese_ls = []
# Enumerate speak_df so that i store the index, row store the value for the rows
for i, row in speak_df.iteritems():
    # use regualre expression to remove special character in this particular column 
    # included open/close parenthesis and comma 
    row = re.sub(r"[\(\),]","",row)
    # split the row into words inside list word_ls
    row_ls = row.split() 
    for word in row_ls:
        if word == 'vietnamese':
            vietnamese_ls.append(i)
print(vietnamese_ls)

[201, 212, 881, 973, 1120, 1146, 1292, 1484, 1689, 1860, 1951, 2246, 2368, 2489, 2801, 2931, 2949, 3668, 3698, 3782, 4101, 4116, 4165, 4404, 4438, 4634, 4970, 5039, 5045, 5169, 5287, 5296, 5381, 5654, 5711, 5839, 6119, 6137, 6428, 6677, 6770, 6883, 7297, 7625, 7755, 7953, 7989, 8034, 8141, 8317, 8560, 8671, 9043, 9422, 9678, 9833, 9844, 10085, 10109, 10178, 10300, 10520, 10658, 10671, 10737, 10814, 10829, 11255, 11758, 11921, 12219, 12282, 12495, 12603, 12740, 12834, 12959, 12964, 13242, 13417, 13452, 13508, 13799, 13812, 13853, 13941, 14048, 14095, 14112, 14114, 14203, 14611, 14753, 14778, 15076, 15195, 15273, 15350, 15575, 15839, 16300, 16409, 16500, 16706, 16849, 16862, 17278, 17279, 17396, 17466, 17526, 17609, 17926, 17959, 18336, 18503, 18567, 18711, 18758, 18815, 18841, 19080, 19081, 19264, 19278, 19387, 19454, 19472, 19607, 19665, 19940, 20164, 20262, 20292, 20838, 21112, 21217, 21276, 21484, 21510, 21628, 21666, 22118, 22123, 22505, 22629, 22798, 23634, 23653, 23691, 23959, 241

In [ ]:
# Total people who speak Vietnamese
print(len(vietnamese_ls))

400


In [ ]:
vietnamese_df = cupid_df.iloc[vietnamese_ls]
vietnamese_df.describe()

,age,income
count,400.000000,400.000000
mean,29.027500,20574.160000
std,6.437028,103507.189542
min,18.000000,-1.000000
25%,25.000000,-1.000000
50%,28.000000,-1.000000
75%,32.000000,-1.000000
max,63.000000,1000000.000000


In [ ]:
# vietnamese_df = cupid_df.iloc[vietnamese_ls]
# vietnamese_f_df = vietnamese_df[vietnamese_df['sex']=='f']
# for index, essay in vietnamese_f_df['essay0'].iteritems():
#     print (index)
#     print (essay)
    

In [ ]:
# for i, v in cupid_df.loc[[1951]].iteritems():
#     print(i)
#     print(v)

In [ ]:
# cupid_df.rename(columns = {
#                         "essay0":"My self summary", 
#                         "essay1":"What I’m doing with my life",
#                         "essay2":"I’m really good at",
#                         "essay3":"The first thing people usually notice about me",
#                         "essay4":"Favorite books, movies, show, music, and food",
#                         "essay5":"The six things I could never do without",
#                         "essay6":"I spend a lot of time thinking about",
#                         "essay7":"On a typical Friday night I am",
#                         "essay8":"The most private thing I am willing to admit",
#                         "essay9":"You should message me if"})

In [5]:
#Combining essays columns
essays_df = cupid_df[['essay0','essay1','essay2','essay3','essay4','essay5','essay6','essay7','essay8','essay9']]
essays_df

NameError: name 'cupid_df' is not defined

In [2]:
essay_0 = essays_df['essay0']
essay_0

NameError: name 'essays_df' is not defined

In [253]:
# for i in range(essays_df.shape[0]): #iterate over rows
#     for j in range(essays_df.shape[1]): #iterate over columns
#         value = essays_df.loc[[i, j]] #get cell value
#         print(value, end="\t")
#     print()

KeyError: 0

In [188]:
#     print (f"YOU'RE AT INDEX {index}")
#     for x in row:
#         print(f"--------{x}")